In [1]:
# BEFORE RUNNING:
# Runtime → Change runtime type → GPU
# Then:
# 1. Run sections 1 & 2
# 2. Upload any image (any name) into:
#    NeuralNeighborStyleTransfer/inputs/content
# 3. Re-run sections 3 onward



In [2]:
# ================================
# 1 Clone repo + clean environment
# ================================

import os
import shutil

# Always start from /content
os.chdir("/content")

# Remove sample_data if present
if os.path.exists("/content/sample_data"):
    shutil.rmtree("/content/sample_data")
    print("sample_data removed")
else:
    print("sample_data already gone")

# Remove repo if it exists (safe re-run)
REPO_PATH = "/content/NeuralNeighborStyleTransfer"
if os.path.exists(REPO_PATH):
    shutil.rmtree(REPO_PATH)
    print("NeuralNeighborStyleTransfer removed")
else:
    print("NeuralNeighborStyleTransfer not present")

!git clone https://github.com/nkolkin13/NeuralNeighborStyleTransfer.git
!pip install torch torchvision numpy imageio matplotlib pillow



sample_data removed
NeuralNeighborStyleTransfer not present
Cloning into 'NeuralNeighborStyleTransfer'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 104 (delta 18), reused 17 (delta 17), pack-reused 83 (from 1)
Receiving objects: 100% (104/104), 27.89 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
# ================================
# 2 Clear defaults + download style
# ================================
import glob

BASE_DIR = "/content/NeuralNeighborStyleTransfer"
INPUTS_DIR = f"{BASE_DIR}/inputs"
CONTENT_DIR = f"{INPUTS_DIR}/content"
STYLE_DIR = f"{INPUTS_DIR}/style"

os.makedirs(CONTENT_DIR, exist_ok=True)
os.makedirs(STYLE_DIR, exist_ok=True)

def clear_folder(folder):
    for f in glob.glob(os.path.join(folder, "*")):
        if os.path.isfile(f):
            os.remove(f)

clear_folder(CONTENT_DIR)
clear_folder(STYLE_DIR)

print("Default content/style images removed")

# Download spaghetti style
STYLE_URL = "https://i.imgur.com/9VbJoDy.jpeg"
STYLE_PATH = f"{STYLE_DIR}/spag.jpeg"

if not os.path.exists(STYLE_PATH):
    !curl -L "$STYLE_URL" -o "$STYLE_PATH"

print("Spaghetti style image ready")



Default content/style images removed
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60292  100 60292    0     0   504k      0 --:--:-- --:--:-- --:--:--  507k
Spaghetti style image ready


In [4]:
# =========================================
# 3 Detect most recent user content image
# =========================================

import os
import glob
import time
import sys

CONTENT_DIR = "/content/NeuralNeighborStyleTransfer/inputs/content"
IMAGE_EXTS = ("*.png", "*.jpg", "*.jpeg", "*.webp")

images = []
for ext in IMAGE_EXTS:
    images.extend(glob.glob(os.path.join(CONTENT_DIR, ext)))

if not images:
    print("ERROR: No content image found.")
    print("Please place an image into:")
    print(CONTENT_DIR)
    print("Then re-run the notebook from Section 3 onward.")
    sys.exit(0)

# Select most recently modified image
selected_content = max(images, key=os.path.getmtime)
original_name = os.path.basename(selected_content)

print("Using content image:")
print(selected_content)
print("Last modified:", time.ctime(os.path.getmtime(selected_content)))



ERROR: No content image found.
Please place an image into:
/content/NeuralNeighborStyleTransfer/inputs/content
Then re-run the notebook from Section 3 onward.


SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# 4 Run Neural Neighbor Style Transfer (recommended spaghetti settings)

# alpha: lower = more spaghetti, higher = more content
# dont_colorize: preserve original content colors
# content_loss: preserve structure more, This creates a shinier look with more highlights for spaghetti
# high_res: higher resolution (more VRAM,
# no_flip: disable patch flipping

# 4 Run Neural Neighbor Style Transfer

if selected_content is None:
    print("Skipping style transfer: no content image.")
else:
    !python /content/NeuralNeighborStyleTransfer/styleTransfer.py \
      --content_path "{selected_content}" \
      --style_path /content/NeuralNeighborStyleTransfer/inputs/style/spag.jpeg \
      --output_path "{RAW_OUTPUT}" \
      --alpha 0.4 \
      --dont_colorize \
      --content_loss






In [ ]:
# 5 Resize output to match original content image size (this could be replaced with a better actual upscaler)

from PIL import Image
import matplotlib.pyplot as plt
import os

if not os.path.exists(RAW_OUTPUT):
    print("Raw NNST output not found:", RAW_OUTPUT)
else:
    # Load original to get size
    orig = Image.open(selected_content)
    orig_w, orig_h = orig.size

    # Load NNST output
    out = Image.open(RAW_OUTPUT)

    # Resize to match original
    out_resized = out.resize((orig_w, orig_h), Image.LANCZOS)
    out_resized.save(FINAL_OUTPUT)

    print("Saved outputs:")
    print("Raw NNST output:", f"raw_{original_name}")
    print("Resized final output:", FINAL_OUTPUT)

    plt.figure(figsize=(6,6))
    plt.imshow(out_resized)
    plt.axis("off")
    print("This is a preview. Download the full-resolution image from the file directory.")

